In [8]:


"""InfoRetrievalTest _Final.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/19mtExjIOR2wl2Abkln_eLZ3GBavIZSCG
"""

import os
import re
import nltk
from nltk.stem import PorterStemmer

import os
import glob
import chardet

def extract_paragraphs(folder_path):
    paragraphs = {}
    count = 1

    for filename in glob.glob(os.path.join(folder_path, '*.txt')):
        with open(filename, 'rb') as file:
            result = chardet.detect(file.read())
            encoding = result['encoding']

        with open(filename, encoding=encoding) as file:
            n1=filename.rfind('\\')
            n1+=1
            n2=filename.rfind('.')
            filenamee=filename[n1:n2:1]
            text = file.read().encode('UTF-8').decode()#extracting text

            file_paragraphs = text.split('\n\n') #extracting paragraphs by splitting text

            for para in file_paragraphs:
                paragraphs[count]=[para,count,filenamee]
                count+=1
    # print("Paras Extracted!!")
    return paragraphs

filepath=r'./dataset'

ulist=extract_paragraphs(filepath)
# print(paras)

text= ""
for value in ulist.values():
    text = text + value[0]
    text = text + " "

# function to split the text into words and convert them all to lowercase (using re library)
# text --> lower case words
def SplitAndLowerText(text):
    #[^A-Za-z\'']+   --> regex for non-aplhabetic chars (and apostrophe) occuring 1 or more times [used as seperator for split]
    words = re.split('[^A-Za-z\']+',text)
    #converting all words to lower case to standardize
    lower_case_words=[word.lower() for word in words]
    return lower_case_words


# function to remove stopwords from list of words and return a list of cleaned words
# lower case words --> cleaned words
def StopwordsRemoval(words):
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stopwords_nltk=set(stopwords.words('english'))
    cleaned_words=[word for word in words if word not in stopwords_nltk]
    return cleaned_words


# function to remove duplicates and return list of unique words
# cleaned words --> unique words
def RemoveDuplicates(words):
    unique_words=[]
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
    return unique_words


# function to perform stemming on words list
# unique words --> stemmed words
def Stemmer(words):
    from nltk.stem import PorterStemmer
    stemmed_words=[]
    for word in words:
        stemmed_words.append(PorterStemmer().stem(word))
    return stemmed_words

#function to create dictiionary from unique and stemmed words
def GenerateDictionaryV2(UniqueWords, StemmedWords, ulist):
    dict={}
    for word in StemmedWords:
        dict[word]=[]
        
    for word in UniqueWords:
      for obj in ulist.values():
        text = ""
        text = text + str(obj[0])
        if word in text.lower():
          x = int(obj[1])
          if x not in dict[PorterStemmer().stem(word)]:
            dict[PorterStemmer().stem(word)].append(x)
    # print("dic v2 generated")
    return dict

def StringMatch(strr, pattern, slen, plen):
 
    # empty pattern (plen=0) can only match with empty string
    if (plen == 0):
        return (slen == 0)
 
    # creating 2D array of 0-M X 0-N to store subsolutions 
    lookup = [[False for i in range(plen + 1)] for j in range(slen + 1)]
 
    # empty pattern matches with empty string
    lookup[0][0] = True
 
    # only '*' can match with empty string
    for j in range(1, plen + 1):
        if (pattern[j - 1] == '*'):
            lookup[0][j] = lookup[0][j - 1]
 
    # filling the table by following the rules below
    for i in range(1, slen + 1):
        for j in range(1, plen + 1):
 
            # if last char of pattern is '*': 
            # (a) either * is zero chars  
            #       --> solution for [i][j-1] is applicable
            # (b) * is a char \
            #       --> solution for [i-1][j] is applicable (since extra char from string is encorporated in *)
            if (pattern[j - 1] == '*'):
                lookup[i][j] = lookup[i][j - 1] or lookup[i - 1][j]
 
            # if last two chars of both are same or last char of pattern is '?', solution for [i-1][j-1] is applicable
            elif ( (pattern[j - 1] == '?') or (strr[i - 1] == pattern[j - 1]) ):
                lookup[i][j] = lookup[i - 1][j - 1]
                
            else:
                lookup[i][j] = False
 
    return lookup[slen][plen]

def EditDistance(str1, str2, len1, len2):
    # creating 2D array of 0-len1 X 0-len2 to store subsolutions    
    dp = [[0 for x in range(len2 + 1)] for x in range(len1 + 1)]
 
    # filling 2D array 
    for i in range(len1 + 1):
        for j in range(len2 + 1):
 
            # if first string is empty, only way is to insert all characters of second string
            if i == 0:
                dp[i][j] = j    # Min. operations = j (length of second string)
 
            # if second string is empty, only way is to remove all characters of first string
            elif j == 0:
                dp[i][j] = i    # Min. operations = i (length of first string)
 
            # if last characters are same, then distance = d[i-1][j-1]
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
 
            # if last characters are different, consider insert(one char short)/remove(one extra char)/replace(diff char)
            else:
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert
                                   dp[i-1][j],        # Remove
                                   dp[i-1][j-1])    # Replace
 
    return dp[len1][len2]

# function to spellcheck a word, returns word/closest word

def SpellCheck(word, VocabList):
    min = 10000
    closest = ""
    for x in VocabList:
        if (StringMatch(x, word, len(x), len(word))):
            return word
        else:
            distance = EditDistance(word, x, len(word), len(x))
            if (distance < min):
                min = distance
                closest = x
    return closest

# defining bitwise functions (using lists)

# bitwise AND
def BitwiseAND(list1, list2):
    bitwise_op = [w1 & w2 for (w1, w2) in zip(list1, list2)]
    return bitwise_op

# bitwise OR
def BitwiseOR(list1, list2):
    bitwise_op = [w1 | w2 for (w1, w2) in zip(list1, list2)]
    return bitwise_op

# bitwise NOT
def BitwiseNOT(list1, list2):
    bitwise_op = [not w1 for w1 in list2]
    bitwise_op = [int(b == True) for b in bitwise_op]
    bitwise_op = [w1 & w2 for (w1,w2) in zip(list1,bitwise_op)]
    return bitwise_op

# EXECUTION
def Execute(q_input):
    if q_input == 1:
      query_input = input("Enter the text or phrase here: \n")
      lower_case_words_query = SplitAndLowerText(query_input)
      cleaned_words_query = StopwordsRemoval(lower_case_words_query)
      unique_words_query = RemoveDuplicates(cleaned_words_query)
#      stemmed_words_query = Stemmer(unique_words_query)
      # if(len(stemmed_words_query)>1):
      #   query = []
      #   for word in stemmed_words_query:
      #       query.append(word)
      #       query.append('and')
      #   query.pop()
      # else:
      #   query = stemmed_words_query
      if(len(unique_words_query)>1):
          query = []
          for word in unique_words_query:
              query.append(word)
              query.append('and')
          query.pop()
      else:
          query = unique_words_query
      # print(query)
    elif q_input == 2:
      query_input = input("Enter the text or phrase here: \n")
      query = query_input.split()
    else:
      print("Selection invalid, Please select again!!")
      return

    #boolean connectors
    connecting_words = []
    #content words
    different_words = []

    # print(query)
    for word in query:
        if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
            different_words.append(word.lower())
        else:
            connecting_words.append(word.lower())

    zeroes_and_ones = []
    zeroes_and_ones_of_all_words = []

    for word in (different_words):
        #if word is normal query
        if word.isalpha():
            closest = SpellCheck(word, unique_words)
            if (word != closest): print("No results have been found for the given word. " + word + " has been corrected. "+ "Showing results for " + "\u0332".join(closest) + " instead")
            stemmed = PorterStemmer().stem(closest)
            # print("Stemmed word being searched:", stemmed)
            # zeroes_and_ones = [0] * (len(ulist)+1)
            zeroes_and_ones = [0] * (len(ulist))           
            for x in dictV2[stemmed]:
              zeroes_and_ones[x] = 1
            zeroes_and_ones_of_all_words.append(zeroes_and_ones)
        
        #if word in wildcary query
        else:
            matches = []
            for x in unique_words:
                if StringMatch(x, word, len(x), len(word)):
                    matches.append(x)

            stemmed_matches=[]
            for x in matches:
                stemmed = PorterStemmer().stem(x)
                if stemmed not in stemmed_matches:
                    stemmed_matches.append(stemmed)

            zeroes_and_ones = [0] * len(ulist)
            temp_zeroes_and_ones = [0] * len(ulist)
            for match in stemmed_matches:
                for x in dictV2[match]:
                    temp_zeroes_and_ones[x] = 1
                zeroes_and_ones = BitwiseOR(zeroes_and_ones, temp_zeroes_and_ones)
            zeroes_and_ones_of_all_words.append(zeroes_and_ones)


    for word in connecting_words:
        # we take the first two words from query (for which the corresponding connecting_word is taken)
        word_list1 = zeroes_and_ones_of_all_words[0]
        word_list2 = zeroes_and_ones_of_all_words[1]

        if word == "and":
            # performs bitwise operation AND on the two lists
            bitwise_op = BitwiseAND(word_list1,word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
        elif word == "or":
            # performs bitwise operation OR on the two lists
            bitwise_op = BitwiseOR(word_list1,word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
        elif word == "not":
            # performs bitwise operation NOT on the two lists
            bitwise_op = BitwiseNOT(word_list1,word_list2)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
    # this results in a final list with bitwise operations done over the entire query

    files_found = []    
    if zeroes_and_ones_of_all_words:
        index_list = zeroes_and_ones_of_all_words[0]
    else:
        index_list = []

    for index in range(len(index_list)):
        if index_list[index]==1:
            files_found.append(ulist.get(index))

    print("\n Files Found:",len(files_found))
    for res in files_found:
      print(res[0])
      print("File ---> " + res[2])
      print("\n")

# preprocessing
lower_case_words = SplitAndLowerText(text)
cleaned_words = StopwordsRemoval(lower_case_words)
unique_words = RemoveDuplicates(cleaned_words)
stemmed_words = Stemmer(unique_words)

dictV2 = GenerateDictionaryV2(unique_words, stemmed_words, ulist)
#dictV2


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
while(1):
  user_input = int(input("Please choose \n 0: Enter zero to exit \n 1:If you want to search by Phrase or a Word \n 2: If you want to search a Wildcard Query \n "))
  if user_input == 0:
    break;
  Execute(user_input)

Please choose 
 0: Enter zero to exit 
 1:If you want to search by Phrase or a Word 
 2: If you want to search a Wildcard Query 
 1
Enter the text or phrase here: 
firee


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


No results have been found for the given word. firee has been corrected. Showing results for f̲i̲r̲e instead

 Files Found: 165
$2,000 for loss by theft, misplacing or losing of
firearms.
File ---> ./dataset/5105072011_booklet


We cover outdoor trees, outdoor shrubs,
outdoor plants or lawns on the residence
premises for loss by the following: fire or
lightning, explosion, riot or civil commotion,
aircraft, vehicles not owned or operated by a
resident of the residence premises, vandalism
or malicious mischief or theft.
File ---> ./dataset/5105072011_booklet


FIRE DEPARTMENT SERVICE CHARGE
File ---> ./dataset/5105072011_booklet


We will pay up to $2,500 for your liability
assumed by contract or agreement for fire
department charges incurred when the fire
department is called to save or protect covered
property from a cause of loss identified under
WHAT LOSSES ARE COVERED - SECTION
|. We do not cover fire department service
charges if’ the property is located within the
limits of the c